In [148]:
import numpy as np
import pandas as pd
import requests
import tqdm
import time
import random
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

In [4]:
url = "https://www.autocasion.com/coches-ocasion"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

session = requests.Session()

response = session.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")
print(response.status_code)

200


In [6]:
anuncios_pagina = soup.find_all("article", class_="anuncio")

total_coches = int(soup.find('div', class_="container listado").find('h1').text.strip().split('\n')[0].replace('.',''))

total_paginas = int(total_coches/len(anuncios_pagina)) + (1 if (total_coches/len(anuncios_pagina))%1 else 0)

total_paginas

6566

In [26]:
def links_coches():
    url_base = "https://www.autocasion.com"
    url_pagina = f"{url_base}/coches-ocasion?direction=desc&sort=updated_at&page="
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    session = requests.Session()
    
    lista_paginas = []
    anuncios_coches = []

    for pagina in range(1, total_paginas +1):
        lista_paginas.append(f'{url_pagina}{pagina}')
        
    lista_prueba = lista_paginas[:1]
        
    for pagina_url in lista_prueba:
        response = session.get(pagina_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        anuncios = soup.find_all("article", class_="anuncio")
        
        for anuncio in anuncios:
            coche_url = anuncio.find("a")['href']
            anuncios_coches.append(f'{url_base}{coche_url}')   
    
        
    return lista_paginas, anuncios_coches

In [28]:
lista_paginas,anuncios_coches = links_coches()
anuncios_coches

['https://www.autocasion.com/coches-segunda-mano/bmw-x3-ocasion/m-sport-166deeef668fdd-ref14271418',
 'https://www.autocasion.com/coches-segunda-mano/dacia-sandero-ocasion/sandero-stepway-tce-expresion-67kw-ref13404343',
 'https://www.autocasion.com/coches-segunda-mano/renault-austral-ocasion/austral-1-3-tce-mild-hybrid-techno-esprit-alpine-cvt-117kw-ref13214707',
 'https://www.autocasion.com/coches-segunda-mano/ford-ecosport-ocasion/ecosport-1-0-ecoboost-active-125-ref12458481',
 'https://www.autocasion.com/coches-segunda-mano/renault-captur-ocasion/captur-blue-dci-zen-edc-85kw-ref13188893',
 'https://www.autocasion.com/coches-segunda-mano/renault-megane-ocasion/megane-1-3-tce-gpf-business-85kw-ref12927462',
 'https://www.autocasion.com/coches-segunda-mano/ford-kuga-ocasion/kuga-1-5tdci-titanium-4x2-120-ref14436080',
 'https://www.autocasion.com/coches-segunda-mano/bmw-serie-6-ocasion/m6-coupe-ref14560922',
 'https://www.autocasion.com/coches-segunda-mano/renault-clio-ocasion/limited-

In [74]:
url_coche = 'https://www.autocasion.com/coches-segunda-mano/citroen-c4-cactus-ocasion/citroen-c4-cactus-monovolumen-82cv-manual-de-5-puertas-ref14463163'
headers = {
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
session = requests.Session()
response = session.get(url_coche, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [78]:
provincia = soup.find_all('div', class_= 'etiquetas')[1].find_all('span')[-1].text

distintivo_ambiental = soup.find_all('div', class_= 'etiquetas')[1].find('img')['alt']

marca_modelo = soup.find('h2').text

detalles_basicos = soup.find('ul', class_= 'datos-basicos-ficha').find_all('li')

lista_detalles_basicos = []

for detalle in detalles_basicos:
    lista_detalles_basicos.append(detalle.text.strip())
    
precio_contado = soup.find('ul', class_= 'tabla-precio').find_all('li')[0].text.replace('\n', '').split(':')[1].replace('.', '').replace('€','').strip()
#precio_nuevo = soup.find('ul', class_= 'tabla-precio').find_all('li')[1].text.replace('\n', '').split('*')[1].strip().replace('.','').split()[0]



In [162]:
lista_detalles_basicos

['Matriculado: 04/2017',
 '89.000 km',
 'Manual',
 'Gasolina',
 'Monovolumen',
 '5 asientos',
 '82 cv',
 '5 Puertas',
 'Violeta']

In [82]:
detalles_carroceria = soup.find('div', class_= 'bloque ficha-tecnica').find('ul', class_= 'tab-spec-1 active').find_all('li')
lista_detalles_carroceria = []
for detalle in detalles_carroceria:
    lista_detalles_carroceria.append(detalle.text.strip().replace('Medidas','').strip())
                                     
lista_detalles_carroceria

AttributeError: 'NoneType' object has no attribute 'find'

In [60]:
detalles_consumos = soup.find('div', class_= 'bloque ficha-tecnica').find('ul', class_= 'tab-spec-2').find_all('li')
lista_detalles_consumo = []

for detalle in detalles_consumos:
    lista_detalles_consumo.append(' '.join(detalle.text.replace('\n','').split()))
     
    
                                     
lista_detalles_consumo

['Consumo medio 0,00 litros/100 km',
 'Consumo en carretera 0,00 litros/100 km',
 'Consumo urbano 0,00 litros/100 km',
 'CO2 (g/km) - g/km',
 'Depósito de combustible (l) 0,00 litros']

In [62]:
detalles_motor = soup.find('div', class_= 'bloque ficha-tecnica').find('ul', class_= 'tab-spec-3').find_all('li')
lista_detalles_motor = []

for detalle in detalles_motor:
    lista_detalles_motor.append(detalle.text.replace('\n', ' ').strip())
    
lista_detalles_motor

['Combustible Corriente eléctrica',
 'Nº de cilindros - cilindros',
 'Cilindrada (cm3) 0',
 'Sobrealimentación -']

In [64]:
detalles_transmision = soup.find('div', class_= 'bloque ficha-tecnica').find('ul', class_= 'tab-spec-4').find_all('li')
lista_detalles_transmision = []

for detalle in detalles_transmision:
    lista_detalles_transmision.append(detalle.text.replace('\n', ' ').strip())
    
lista_detalles_transmision

['Tipo de transmisión Directo, sin caja de cambios',
 'TracciónTracción delantera',
 'Número de marchas1']

In [66]:
detalles_prestaciones = soup.find('div', class_= 'bloque ficha-tecnica').find('ul', class_= 'tab-spec-5').find_all('li')
lista_detalles_prestaciones = []

for detalle in detalles_prestaciones:
    lista_detalles_prestaciones.append(detalle.text)
    
lista_detalles_prestaciones

['Potencia (KW)140',
 'Potencia (CV)190',
 'Par (Nm)375',
 'Velocidad máxima (km/h)160',
 'Aceleración 0-100 km/h (s)9,20']

In [84]:
lista_detalles_vendedor = soup.find('div', class_= 'datos-concesionario').find('p').text.strip().split('\n')

nombre_vendedor = lista_detalles_vendedor[0]
tipo_vendedor = lista_detalles_vendedor[1].strip()

In [86]:
nombre_vendedor

'Dalida'

In [88]:
tipo_vendedor

'Particular - Málaga'

In [146]:
ficha_tecnica = soup.find('div', class_= 'bloque no-eurotax').find_all('ul')
lista_ficha_tecnica = []

for detalle in range(0,len(ficha_tecnica)):
    lista_ficha_tecnica.append(soup.find('div', class_= 'bloque no-eurotax').find_all('ul')[detalle].text)
    
    
lista_ficha_tecnica    

['\nCarrocería\nCarrocería:  Monovolumen\nNúmero de puertas:  5 Puertas\nNúmero de plazas:              \n                                                    5\n                                    \n     Plazas\n',
 '\nMotor\n\n                    Combustible: \n                    \n                                                                            Gasolina\n                                            \n\nPotencia:              \n                                        82\n                                                \n    \n',
 '\nTransmisión\nTipo de cambio:  Manual\n']

In [150]:
info_coche = {}

# Función para buscar y extraer un valor específico de cada cadena
def buscar_detalle(texto, campo):
    # Usamos expresiones regulares para capturar el campo deseado seguido de su valor
    pattern = rf"{campo}:?\s+([^\n]+)"
    match = re.search(pattern, texto)
    return match.group(1).strip() if match else None


In [158]:
for detalle in lista_ficha_tecnica:
    if "Carrocería" in detalle:
        info_coche['carroceria'] = buscar_detalle(detalle, "Carrocería:")
        info_coche['numero_puertas'] = buscar_detalle(detalle, "Número de puertas:")
        info_coche['numero_plazas'] = buscar_detalle(detalle, "Número de plazas:")
        info_coche['peso'] = buscar_detalle(detalle, "Peso:")
        
    elif "Motor" in detalle:
        info_coche['combustible'] = buscar_detalle(detalle, "Combustible:")
        info_coche['potencia'] = buscar_detalle(detalle, "Potencia:")
        info_coche['cilindrada'] = buscar_detalle(detalle, "Cilindrada:")
    elif "Transmisión" in detalle:
        info_coche['tipo_cambio'] = buscar_detalle(detalle, "Tipo de cambio:")
        info_coche['marchas'] = buscar_detalle(detalle, "Marchas:")
        
        
print(info_coche)

{'carroceria': 'Monovolumen', 'numero_puertas': '5 Puertas', 'numero_plazas': '5', 'combustible': 'Gasolina', 'potencia': '82', 'tipo_cambio': 'Manual'}


In [154]:
info_coche['numero_puertas']

'5 Puertas'

In [ ]:
.strip()
                               .replace('Carrocería\n','').replace('Consumos y emisiones\n','').replace('Motor\n','').replace('Transmisión\n','').replace('Prestaciones\n','').replace(' \n ','')
                               .replace('  \n','').replace(' \n ','')
                              .strip())